# ST-file
Demo to show how to load, modify and save a st file.

In [ ]:
from lacbox.io import load_st, save_st
from lacbox.test import test_data_path
import matplotlib.pyplot as plt

## Load a ST-file (`load_st`)

In [ ]:
st_path = test_data_path + '/dtu_10_mw/data/DTU_10MW_RWT_Blade_st.dat'
st_data = load_st(st_path)
print(f"Number of sets: len(st_data)={len(st_data)}")
print(f"Number of subsets: len(st_data[0])={len(st_data[0])}")
print(f"Subset keys: st_data[i][j].keys()={st_data[0][0].keys()}")

## Modify the data

In [ ]:
# Up scaling 
R0 = 89.16 # [m] Baseline rotor diameter
R1 = 98.0 # [m] New rotor size
s = R1/R0 # [-] Scaling factor

for iset, set in enumerate(st_data):
    for jsubset, subset in enumerate(set):
        # s^1
        for name in ['s', 'x_cg', 'y_cg', 'ri_x', 'ri_y', 'x_sh', 'y_sh', 'x_e', 'y_e']:
            st_data[iset][jsubset][name] = subset[name]*s**1
        # s^2
        for name in ['m', 'A']:
            st_data[iset][jsubset][name] = subset[name]*s**2
        # s^4
        for name in ['I_x', 'I_y', 'I_p']:
            st_data[iset][jsubset][name] = subset[name]*s**4

## Save the modified PC file (`save_st`)

In [ ]:
# Saving the upscaled data
st_path = "DTU_10MW_RWT_Blade_st_DEMO.dat"
save_st(st_path, st_data)
# Saving only a single subset
save_st(st_path, st_data[0][1])
# Saving multiple sets (dummy example)
save_st(st_path, [
    st_data[0], # Set one
    [ # Set two
        st_data[0][0], # Subset 1
        st_data[0][1], # Subset 2
        st_data[0][1], # Subset 3 (dummy same as subset 2)
    ]
    ]
)

## Visualize the data

In [ ]:
# Reload the data from the original st file
st_path = test_data_path + "/dtu_10_mw/data/DTU_10MW_RWT_Blade_st.dat"
st_data_org = load_st(st_path)

# Create figure
fig, axs = plt.subplots(3, 1)#, figsize=(7, 6))

# Extract subset
subset = st_data[0][0]
subset_org = st_data_org[0][0]

# Plotting scaled data
axs[0].plot(subset["s"], subset["x_cg"], color="C0", label="Scaled")
axs[1].plot(subset["s"], subset["m"], color="C0", label="Scaled")
axs[2].plot(subset["s"], subset["I_x"], color="C0", label="Scaled")

# Plotting original data
axs[0].plot(subset_org["s"], subset_org["x_cg"], "--", color="C0", label="Original")
axs[1].plot(subset_org["s"], subset_org["m"], "--", color="C0", label="Original")
axs[2].plot(subset_org["s"], subset_org["I_x"], "--", color="C0", label="Original")

# Adding legend and axis labels
loc = 1
axs[0].legend(loc=loc, title="$s^1$", ncol=1, framealpha=1)
axs[1].legend(loc=loc, title="$s^2$", ncol=1, framealpha=1)
axs[2].legend(loc=loc, title="$s^4$", ncol=1, framealpha=1)
axs[0].set_ylabel("$x_{cg}$ [m]")
axs[1].set_ylabel("$m$ [kg/m]")
axs[2].set_ylabel("$I_x$ [m$^4$]")
axs[2].set_xlabel(r"Blade curve length ($s$) [m]")
fig.tight_layout()